In [1]:
# Tabla de datos

import pandas as pd
import os

dataBol=pd.read_excel(os.path.join('data','dataBolivia_indicadores.xlsx'))
dataBol.head()

,DEPARTAMENTO,PROVINCIA,TOTAL_VIVIENDAS,TAMAÑO_PROMEDIO_DEL_HOGAR,PORCENTAJE_ DE_VIVIENDAS_CON_ENERGIA_ELECTRICA,PERSONAS_CON_DIFICULTADES_PERMANENTES
0,CHUQUISACA,Oropeza,77376,3.62,87.08,10829
1,CHUQUISACA,Azurduy,5956,4.11,27.50,983
2,CHUQUISACA,Zudáñez,9116,4.21,48.96,1156
3,CHUQUISACA,Tomina,9217,3.77,46.23,1640
4,CHUQUISACA,Hernando Siles,8158,3.84,63.67,1554


In [3]:
# Mapa

os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd

dataBolmap=gpd.read_file(os.path.join('maps','Provincias.zip'))

dataBolmap.head()

CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)

In [ ]:
# Preprocessing
capitalizeColumns=lambda x: x.str.upper().str.strip()
dataBol[['DEPARTAMENTO','PROVINCIA']]=dataBol[['DEPARTAMENTO','PROVINCIA']].apply(capitalizeColumns)
dataBolmap[['DEPARTAMEN','PROVINCIA']]=dataBolmap[['DEPARTAMEN','PROVINCIA']].apply(capitalizeColumns)

In [ ]:
import unidecode

byePunctuation=lambda x: unidecode.unidecode(x)
dataBol[['DEPARTAMENTO','PROVINCIA']]=dataBol[['DEPARTAMENTO','PROVINCIA']].applymap(byePunctuation)
dataBolmap[['DEPARTAMEN','PROVINCIA']]=dataBolmap[['DEPARTAMEN','PROVINCIA']].applymap(byePunctuation)

In [ ]:
len(dataBol.PROVINCIA),len(dataBolmap.PROVINCIA)

In [ ]:
dataBol.PROVINCIA.duplicated().sum(),dataBolmap.PROVINCIA.duplicated().sum()

In [ ]:
# concatenating

dataBol['provDist']=["+".join(pd) for pd in zip (dataBol.DEPARTAMENTO,dataBol.PROVINCIA)]
dataBolmap['provDist']=["+".join(pd) for pd in zip (dataBolmap.DEPARTAMEN,dataBolmap.PROVINCIA)]

In [ ]:
# the new column looks like this:

dataBol['provDist'].head()

In [ ]:
# replacing dashes and multiple spaces by a simple space

dataBol.provDist=dataBol.provDist.str.replace("\-|\_|\s+"," ",regex=True)
dataBolmap.provDist=dataBolmap.provDist.str.replace("\-|\_|\s+"," ",regex=True)

In [ ]:
dataBol

In [ ]:
dataBolmap

In [ ]:
# EJERCICIO: 1

In [ ]:
nomatch_df=set(dataBol.provDist)- set(dataBolmap.provDist)
nomatch_gdf=set(dataBolmap.provDist)-set(dataBol.provDist) 

In [ ]:
len(nomatch_df), len(nomatch_gdf)

In [ ]:
# pick the closest match from nomatch_gdf for a value in nomatch_df

from thefuzz import process

[(dis,process.extractOne(dis,nomatch_gdf)) for dis in sorted(nomatch_df)]

In [ ]:
# is this OK?

{dis:process.extractOne(dis,nomatch_gdf)[0] for dis in sorted(nomatch_df)}

In [ ]:
# then:

changes_df={dis:process.extractOne(dis,nomatch_gdf)[0] for dis in sorted(nomatch_df)}

In [ ]:
# Now, make the replacements:

dataBol.provDist.replace(changes_df,inplace=True)

In [ ]:
# Now the merge can happen:

dataBolmap=dataBolmap.merge(dataBol, on='provDist')

In [ ]:
# check

dataBolmap.info()

In [ ]:
# We can get rid of some columns:

bye=['id','c_ut','DEPARTAMENTO', 'PROVINCIA_y']
dataBolmap.drop(columns=bye,inplace=True)

# keeping
dataBolmap.head()

In [ ]:
# Cambiamos los nombres para que sean más cómodos de usar
dataBolmap.rename(columns={'Provincia_x': 'Provincias'}, inplace=True)
dataBolmap=dataBolmap.rename(columns={'DEPARTAMEN': 'DEPARTAMENTO'})
dataBolmap=dataBolmap.rename(columns={'PROVINCIA_x': 'PROVINCIA'})
dataBolmap.head()